In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
from functions  import intensity_data
from scipy.io import loadmat
sub=['F1','F2','M1','M2']
# sub = ['M2']
fold=['Fold1','Fold2','Fold3','Fold4']
# fold = ['Fold1']
seg = 'Upper'
a = list(range(99,100))
b = [99.25,99.5,99.75,100]
per = a + b
for p in per:
    for s in sub:
        mat = loadmat(seg+'/'+s+'/'+"pxl_np_"+str(p)+".mat")
        np_fold = mat['pxl_np']
        for f in fold: 
            print('perc'+str(p)+'Subject'+s+'and'+f)
            #dataset = pandas.read_csv(seg+'/'+s+'/'+f+'/'+"Grid_Wise_Set_train_"+str(p)+".csv", delimiter=",",header = None)
            #dataset.replace(numpy.inf, numpy.nan)
            #dataset[dataset==numpy.inf] = numpy.nan
            #dataset.fillna(dataset.mean(), inplace=True)
            #dataset = dataset.as_matrix(columns = None)
            ind = fold.index(f)
            np = np_fold[0,ind][1][0][0]
            dataset = loadmat(seg+'/'+s+'/'+f+'/'+"Intensity_Set_"+str(p)+".mat")
            a1= dataset['intensity_profile_frame']
            l = a1.shape[1]
            l1 = int(l/4)
            a = numpy.array(a1[0][0:l-l1])
            a = a.reshape(1,l-l1)
            b = numpy.array(a1[0][l-l1:l])
            b = b.reshape(1,l1)
            a = intensity_data(a)
            b = intensity_data(b)
            print(len(a))
            print(len(b))
            X = a[:,0:np+1]
            Xorg = X.copy()
            X = X[~numpy.isnan(X).any(axis=1)]
            Y = a[:,np+1]
            Y = Y[~numpy.isnan(Xorg).any(axis=1)]
            X1 = b[:,0:np+1]
            X1org = X1.copy()
            X1 = X1[~numpy.isnan(X1).any(axis=1)]
            Y1 = b[:,np+1]
            Y1 = Y1[~numpy.isnan(X1org).any(axis=1)]
            if len(a)<=50000:
                n1 = 64
                n2 = 64
                n3 = 64
                bs = 50
            elif len(a)<=100000:
                n1 = 128
                n2 = 128
                n3 = 128
                bs = 100
            elif len(a)<=200000:
                n1 = 256
                n2 = 128
                n3 = 128
                bs = 150
            elif len(a)<=300000:
                n1 = 256
                n2 = 256
                n3 = 128
                bs = 200
            elif len(a)<=400000:
                n1 = 256
                n2 = 256
                n3 = 256
                bs = 250
            else:
                n1 = 512
                n2 = 256
                n3 = 256
                bs = 300
#             if len(a)<=40000:
#                 n1 = 64
#                 n2 = 64
#                 n3 = 64
#             else:
#                 n1 = 128
#                 n2 = 128
#                 n3 = 128
            def base_model():
                model = Sequential()
                model.add(Dense(n1,activation="relu", kernel_initializer="normal", input_dim=np+1))
                model.add(Dropout(0.15))
#                 if l > 20000:
                model.add(Dense(n2, activation="relu", kernel_initializer="normal"))
                model.add(Dropout(0.15))
#                 if l > 50000:
                model.add(Dense(n3, activation="relu", kernel_initializer="normal"))
                model.add(Dropout(0.15))  
                model.add(Dense(1, activation="relu", kernel_initializer="normal"))
                model.compile(loss='mean_squared_error',optimizer='adam')
                return model
            fname=(seg+'/'+s+'/'+f+'/'+'Model_GridWise_dev_' + str(p) + '.hdf5')
            callbacks = [EarlyStopping(monitor='val_loss', patience=4, min_delta = 0.00001), ModelCheckpoint(fname, monitor='val_loss', verbose=0, save_best_only=True, mode='auto', period=1)]
            estimator = KerasRegressor( build_fn = base_model)
            estimator.fit(X, Y, validation_data=(X1,Y1), epochs=100, batch_size=bs, callbacks=callbacks)
            a = []
            b = []
            X1 = []
            Y1 = []
            X1org = []
            Xorg = []
            X = []
            Y = []
            a1 = []
            dataset = []            
print("TRAINING COMPLETE!!")

Using TensorFlow backend.


perc99SubjectF1andFold1
228888
73562
Train on 204569 samples, validate on 66343 samples
Epoch 1/100
204569/204569 [==============================] - 7s 36us/step - loss: 0.0434 - val_loss: 0.0394
Epoch 2/100
204569/204569 [==============================] - 7s 36us/step - loss: 0.0360 - val_loss: 0.0386
Epoch 3/100
204569/204569 [==============================] - 8s 40us/step - loss: 0.0340 - val_loss: 0.0369
Epoch 4/100
204569/204569 [==============================] - 9s 42us/step - loss: 0.0325 - val_loss: 0.0350
Epoch 5/100
204569/204569 [==============================] - 9s 46us/step - loss: 0.0314 - val_loss: 0.0351
Epoch 6/100
204569/204569 [==============================] - 9s 44us/step - loss: 0.0307 - val_loss: 0.0339
Epoch 7/100
204569/204569 [==============================] - 9s 46us/step - loss: 0.0300 - val_loss: 0.0341
Epoch 8/100
204569/204569 [==============================] - 9s 45us/step - loss: 0.0295 - val_loss: 0.0328
Epoch 9/100
204569/204569 [=====================

169215/169215 [==============================] - 9s 52us/step - loss: 0.0303 - val_loss: 0.0298
Epoch 19/100
169215/169215 [==============================] - 9s 51us/step - loss: 0.0298 - val_loss: 0.0289
Epoch 20/100
169215/169215 [==============================] - 8s 50us/step - loss: 0.0298 - val_loss: 0.0292
Epoch 21/100
169215/169215 [==============================] - 9s 52us/step - loss: 0.0295 - val_loss: 0.0295
Epoch 22/100
169215/169215 [==============================] - 9s 52us/step - loss: 0.0294 - val_loss: 0.0303
Epoch 23/100
169215/169215 [==============================] - 9s 54us/step - loss: 0.0292 - val_loss: 0.0298
perc99SubjectF1andFold4
190603
64006
Train on 94769 samples, validate on 31363 samples
Epoch 1/100
94769/94769 [==============================] - 6s 60us/step - loss: 0.0521 - val_loss: 0.0439
Epoch 2/100
94769/94769 [==============================] - 5s 48us/step - loss: 0.0380 - val_loss: 0.0401
Epoch 3/100
94769/94769 [==============================] - 5

Epoch 8/100
163143/163143 [==============================] - 13s 79us/step - loss: 0.0316 - val_loss: 0.0291
Epoch 9/100
163143/163143 [==============================] - 13s 78us/step - loss: 0.0313 - val_loss: 0.0301
Epoch 10/100
163143/163143 [==============================] - 13s 81us/step - loss: 0.0308 - val_loss: 0.0299
Epoch 11/100
163143/163143 [==============================] - 13s 80us/step - loss: 0.0304 - val_loss: 0.0286
Epoch 12/100
163143/163143 [==============================] - 13s 82us/step - loss: 0.0301 - val_loss: 0.0282
Epoch 13/100
163143/163143 [==============================] - 13s 81us/step - loss: 0.0298 - val_loss: 0.0286
Epoch 14/100
163143/163143 [==============================] - 13s 80us/step - loss: 0.0294 - val_loss: 0.0284
Epoch 15/100
163143/163143 [==============================] - 13s 82us/step - loss: 0.0292 - val_loss: 0.0277
Epoch 16/100
163143/163143 [==============================] - 13s 81us/step - loss: 0.0290 - val_loss: 0.0274
Epoch 17/100

122857/122857 [==============================] - 9s 76us/step - loss: 0.0260 - val_loss: 0.0315
Epoch 26/100
122857/122857 [==============================] - 9s 75us/step - loss: 0.0259 - val_loss: 0.0304
Epoch 27/100
122857/122857 [==============================] - 9s 76us/step - loss: 0.0257 - val_loss: 0.0304
perc99SubjectM1andFold1
264015
88370
Train on 222741 samples, validate on 75072 samples
Epoch 1/100
222741/222741 [==============================] - 18s 80us/step - loss: 0.0545 - val_loss: 0.0449
Epoch 2/100
222741/222741 [==============================] - 17s 76us/step - loss: 0.0443 - val_loss: 0.0401
Epoch 3/100
222741/222741 [==============================] - 17s 76us/step - loss: 0.0418 - val_loss: 0.0385
Epoch 4/100
222741/222741 [==============================] - 17s 76us/step - loss: 0.0403 - val_loss: 0.0368
Epoch 5/100
222741/222741 [==============================] - 17s 76us/step - loss: 0.0390 - val_loss: 0.0361
Epoch 6/100
222741/222741 [==========================

...

143933/143933 [==============================] - 18s 124us/step - loss: 0.0254 - val_loss: 0.0255
perc100SubjectF2andFold2
274150
93361
Train on 165880 samples, validate on 57218 samples
Epoch 1/100
165880/165880 [==============================] - 24s 144us/step - loss: 0.0450 - val_loss: 0.0333
Epoch 2/100
165880/165880 [==============================] - 17s 101us/step - loss: 0.0348 - val_loss: 0.0308
Epoch 3/100
165880/165880 [==============================] - 17s 103us/step - loss: 0.0331 - val_loss: 0.0311
Epoch 4/100
165880/165880 [==============================] - 16s 97us/step - loss: 0.0321 - val_loss: 0.0296
Epoch 5/100
165880/165880 [==============================] - 17s 100us/step - loss: 0.0314 - val_loss: 0.0290
Epoch 6/100
165880/165880 [==============================] - 16s 97us/step - loss: 0.0306 - val_loss: 0.0287
Epoch 7/100
165880/165880 [==============================] - 17s 105us/step - loss: 0.0301 - val_loss: 0.0283
Epoch 8/100
165880/165880 [==================

Epoch 27/100
124346/124346 [==============================] - 13s 105us/step - loss: 0.0233 - val_loss: 0.0271
Epoch 28/100
124346/124346 [==============================] - 14s 114us/step - loss: 0.0232 - val_loss: 0.0272
Epoch 29/100
124346/124346 [==============================] - 14s 116us/step - loss: 0.0230 - val_loss: 0.0273
Epoch 30/100
124346/124346 [==============================] - 14s 114us/step - loss: 0.0230 - val_loss: 0.0279
Epoch 31/100
124346/124346 [==============================] - 14s 114us/step - loss: 0.0227 - val_loss: 0.0275
perc100SubjectM1andFold1
312801
104641
Train on 263856 samples, validate on 88904 samples
Epoch 1/100
263856/263856 [==============================] - 36s 136us/step - loss: 0.0462 - val_loss: 0.0380
Epoch 2/100
263856/263856 [==============================] - 30s 114us/step - loss: 0.0366 - val_loss: 0.0325
Epoch 3/100
263856/263856 [==============================] - 31s 119us/step - loss: 0.0349 - val_loss: 0.0308
Epoch 4/100
263856/263856

169162/169162 [==============================] - 26s 154us/step - loss: 0.0477 - val_loss: 0.0317
Epoch 2/100
169162/169162 [==============================] - 20s 120us/step - loss: 0.0362 - val_loss: 0.0288
Epoch 3/100
169162/169162 [==============================] - 20s 119us/step - loss: 0.0334 - val_loss: 0.0268
Epoch 4/100
169162/169162 [==============================] - 20s 118us/step - loss: 0.0317 - val_loss: 0.0258
Epoch 5/100
169162/169162 [==============================] - 20s 119us/step - loss: 0.0304 - val_loss: 0.0244
Epoch 6/100
169162/169162 [==============================] - 20s 121us/step - loss: 0.0296 - val_loss: 0.0238
Epoch 7/100
169162/169162 [==============================] - 20s 116us/step - loss: 0.0290 - val_loss: 0.0241
Epoch 8/100
169162/169162 [==============================] - 20s 121us/step - loss: 0.0285 - val_loss: 0.0239
Epoch 9/100
169162/169162 [==============================] - 21s 122us/step - loss: 0.0278 - val_loss: 0.0228
Epoch 10/100
169162/16

237993/237993 [==============================] - 28s 118us/step - loss: 0.0259 - val_loss: 0.0262
Epoch 24/100
237993/237993 [==============================] - 29s 120us/step - loss: 0.0257 - val_loss: 0.0262
Epoch 25/100
237993/237993 [==============================] - 29s 121us/step - loss: 0.0255 - val_loss: 0.0260
Epoch 26/100
237993/237993 [==============================] - 28s 119us/step - loss: 0.0253 - val_loss: 0.0261
Epoch 27/100
237993/237993 [==============================] - 28s 118us/step - loss: 0.0252 - val_loss: 0.0258
Epoch 28/100
237993/237993 [==============================] - 29s 123us/step - loss: 0.0251 - val_loss: 0.0260
Epoch 29/100
237993/237993 [==============================] - 28s 118us/step - loss: 0.0250 - val_loss: 0.0256
Epoch 30/100
237993/237993 [==============================] - 29s 120us/step - loss: 0.0249 - val_loss: 0.0257
Epoch 31/100
237993/237993 [==============================] - 28s 118us/step - loss: 0.0247 - val_loss: 0.0254
Epoch 32/100
2

161361/161361 [==============================] - 20s 122us/step - loss: 0.0278 - val_loss: 0.0260
Epoch 13/100
161361/161361 [==============================] - 20s 122us/step - loss: 0.0273 - val_loss: 0.0261
Epoch 14/100
161361/161361 [==============================] - 20s 125us/step - loss: 0.0269 - val_loss: 0.0269
Epoch 15/100
161361/161361 [==============================] - 22s 137us/step - loss: 0.0266 - val_loss: 0.0264
Epoch 16/100
161361/161361 [==============================] - 22s 136us/step - loss: 0.0264 - val_loss: 0.0255
Epoch 17/100
161361/161361 [==============================] - 22s 134us/step - loss: 0.0260 - val_loss: 0.0253
Epoch 18/100
161361/161361 [==============================] - 23s 140us/step - loss: 0.0256 - val_loss: 0.0250
Epoch 19/100
161361/161361 [==============================] - 25s 158us/step - loss: 0.0254 - val_loss: 0.0244
Epoch 20/100
161361/161361 [==============================] - 24s 148us/step - loss: 0.0252 - val_loss: 0.0241
Epoch 21/100
1

In [24]:
print(X.shape)
X = X[~numpy.isnan(X).any(axis=1)]

(12957, 3)


In [25]:
X.shape

(12048, 3)

In [5]:
numpy.isnan(X).any(axis=1)

array([False, False, False, ..., False, False, False])